In [1]:
import pandas as pd
import numpy as np
import sys
# from torch.utils.data import DataLoader

from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer

from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AdamW
import torch

from statsmodels.stats.contingency_tables import mcnemar
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

/home/ahaque2/venv/py3_7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('dataset/MFTC_full_dataset.csv')
# data = pd.read_csv('dataset/MFTC_clean.csv')

In [3]:
data = data.drop_duplicates()
data['label_id'] = pd.factorize(data.label)[0]
# data = data[((data.label_id != 11) & (data.label_id != -1))].dropna(subset = ['text'])
# data.to_csv('dataset/MFTC_clean.csv')

In [4]:
data.shape, data.columns

((33679, 8),
 Index(['Unnamed: 0', 'Unnamed: 0.1', 'ids', 'text', 'top', 'label',
        'clean_text', 'label_id'],
       dtype='object'))

In [5]:
data.label.value_counts()

non-moral      14867
harm            3332
cheating        2972
care            2257
fairness        2118
loyalty         1875
subversion      1700
authority       1496
degradation     1286
betrayal        1119
purity           657
Name: label, dtype: int64

In [6]:
data.label_id.value_counts()

4     14867
3      3332
2      2972
0      2257
5      2118
8      1875
1      1700
7      1496
10     1286
6      1119
9       657
Name: label_id, dtype: int64

In [7]:
data = data.dropna(subset = ['clean_text'])
data.clean_text.isna().sum()

0

In [8]:
def preprocess_data(data):
    
    #Removes Numbers
    data = data.astype(str).str.replace('\d+', '')
    lower_text = data.str.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer =  TweetTokenizer()

    def lemmatize_text(text):
        return [(lemmatizer.lemmatize(w)) for w \
                       in w_tokenizer.tokenize((text))]
    
    def preprocess(txt):

        stop_words = set(stopwords.words('english'))
        exclude_words = ['donald', 'trump', 'joe', 'biden', 'president']
        return [w for w in txt if (not w.lower() in stop_words and not w.lower() in exclude_words)]

        # # print(txt)
        # # sys.exit()
        # proc_sent = []
        # # print(txt)
        # # sys.exit()
        # for line in txt:
        #     # word_tokens = word_tokenize(line)
        #     print(line)
#             sent = [w for w in line if (not w.lower() in stop_words and not w.lower() in exclude_words)]
#             # pd.concat((proc_sent, pd.Series(sent)), axis = 0)
#             proc_sent.append(sent)
#             print(proc_sent)
#             sys.exit()

#         return proc_sent
    
    def remove_punctuation(words):
        new_words = []
        # print(words)
        # sys.exit()
        for word in words:
          new_word = re.sub(r'[^\w\s]', '', (word))
          if new_word != '':
             new_words.append(new_word)
        # print(new_words)
        # sys.exit()
        
        return new_words
    
    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    words = words.apply(preprocess)
    
    # print(words)
    # sys.exit()
    
    return pd.DataFrame(words)

In [9]:
data.shape, data.columns

((33419, 8),
 Index(['Unnamed: 0', 'Unnamed: 0.1', 'ids', 'text', 'top', 'label',
        'clean_text', 'label_id'],
       dtype='object'))

In [10]:
# data[['label', 'label_id']].drop_duplicates().to_csv('dataset/MFT_label_to_id_mapping.csv')

In [11]:
X = np.array(data.text)
y = np.array(data.label_id)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.text, y,
                                                    stratify=y, 
                                                    test_size=0.20)

In [12]:
data = data.dropna(subset = ['clean_text'])

In [13]:
data = data.sample(frac=1)

In [14]:
data.index = list(range(data.shape[0]))

In [15]:
# data.index = list(range(data.shape[0]))
# # data.index

In [16]:
# !pip3 install -U huggingface_hub

In [17]:
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base', use_fast=True)
# tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base', use_fast=True)

# model = AutoModel.from_pretrained('vinai/bertweet-base', num_labels=11)
# model = AutoModelForSequenceClassification.from_pretrained('vinai/bertweet-base', num_labels=11)
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base', num_labels=11)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and

In [18]:
# train_encodings = tokenizer(X_train.tolist(), truncation=True, max_length=500, padding=True)
# test_encodings = tokenizer(X_test.tolist(), truncation=True, max_length=500, padding=True)

data_encodings = tokenizer(data.clean_text.tolist(), truncation=True, max_length=130, padding=True)

In [19]:
# data_encodings

In [20]:
np.array(data.clean_text.tolist()).shape

(33419,)

In [21]:
data.label_id.nunique()

11

In [22]:
import torch

class setDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# train_dataset = setDataset(train_encodings, y_train)
# test_dataset = setDataset(test_encodings, y_test)

dataset = setDataset(data_encodings, data.label_id)

In [23]:
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

def get_pytorch_model(model, train_dataset):

    device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
    # device = torch.device('cpu')

    model.to(device)
    model.train()

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

    optim = AdamW(model.parameters(), lr=5e-5)

    for epoch in range(3):
        for batch in train_loader:
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # print(input_ids.size(), attention_mask.shape, labels.shape)
            # sys.exit()
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            loss.backward()
            optim.step()

    model.eval()
    
    return model

In [24]:
model = get_pytorch_model(model, dataset)

In [25]:
sys.exit()

SystemExit: 

In [33]:
# import pickle

# model.save_pretrained("model/model_MFT_all")
# pickle.dump(topics, open( "bertopic/topics.pickle", "wb" ) )
# assert topics == new_topics

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base', use_fast=True)
model2 = AutoModelForSequenceClassification.from_pretrained("model/model_MFT_all", num_labels=11).to(device)

In [29]:
device

device(type='cuda')

In [30]:
def get_predictions(X_test, tokenizer, model):
    
    x = 0
    pred_labs = []
    while x < X_test.shape[0]:
        #model = DistilBertForSequenceClassification.from_pretrained("models/DistilBert_1", num_labels=4).to(device)
        device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
        # y = x + 100
        if((x+10) < X_test.shape[0]):
            y = x + 10
        else:
            y = X_test.shape[0]

        x_test = X_test.iloc[x:y].tolist()
        # print(x, y, len(x_test))
        inputs = tokenizer(x_test,  padding=True, truncation=True, max_length=500, return_tensors="pt").to(device)
        outputs = model(**inputs)

        #softmax_labs = [(round(x.tolist()[0], 4), round(x.tolist()[1], 4)) for x in outputs[0].softmax(1)]
        y_pred = [int(x.argmax()) for x in outputs[0].softmax(1)]
        pred_labs.extend(y_pred)
        x+=10
        
    return pred_labs

In [31]:
y_pred = get_predictions(X_test, tokenizer, model)
len(y_pred)

6684

In [32]:
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.69      0.73       451
           1       0.58      0.43      0.49       339
           2       0.54      0.82      0.65       594
           3       0.70      0.71      0.71       666
           4       0.87      0.83      0.85      2927
           5       0.72      0.74      0.73       423
           6       0.47      0.33      0.39       224
           7       0.60      0.68      0.64       299
           8       0.86      0.55      0.67       373
           9       0.54      0.74      0.62       131
          10       0.55      0.62      0.58       257

    accuracy                           0.73      6684
   macro avg       0.66      0.65      0.64      6684
weighted avg       0.75      0.73      0.74      6684

[[ 311    2   10   31   43    9    2   17    6    8   12]
 [   1  146   59   16   47    4   33   16    0    0   17]
 [   0    8  488   23   36   14   11    3    0    0   11]
 [  10    7  